In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import math

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric

from  utils import run_experiment
from models import GCN, iterativeGCN
from torch_geometric.datasets import Planetoid

In [2]:
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

In [3]:
list(data.x.shape)

[2708, 1433]

In [21]:
def add_noise(data, percent=0.7):
    #add random 1's to data
    if percent >= 0 and percent <= 1:
        len = np.prod(list(data.x.shape))
        ones = math.floor(len * percent)
        zeros = len - ones
        noise = torch.cat((torch.zeros(zeros), torch.ones(ones)))
        noise = noise[torch.randperm(noise.size(0))]
        noise = torch.reshape(noise, data.x.shape)
        data.x += noise
    return data

In [25]:
hid_dim = 16
num_iter = 2
smooth_fac = 0.7

In [29]:
model = iterativeGCN(input_dim=dataset.num_features,
                                output_dim=dataset.num_classes,
                                hidden_dim=hid_dim,
                                num_train_iter=num_iter,
                                smooth_fac=smooth_fac,
                                schedule=None,
                                dropout=0.5,
                                xavier_init=True
                                )

In [34]:
model.parameters

<bound method Module.parameters of iterativeGCN(
  (encoder): Linear(in_features=1433, out_features=16, bias=True)
  (gc): GCNConv(16, 16)
  (decoder): Linear(in_features=16, out_features=7, bias=True)
)>

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)